# Creating a Simple 'Agent' (not really) with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

True

In [2]:

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [3]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

### Runner Class
The Runner class from the agents library is the executor/orchestrator that runs your AI agent and manages the interaction with the language model. 

It has two main methods:

**1. Runner.run() - Synchronous/Complete Response**

Executes the agent and waits for the complete response:

```Python
result = await Runner.run(nutrition_agent, "How healthy are bananas?")
```

**Returns**: A RunResult object containing:
- The complete final output
- Agent metadata
- Raw responses from the model
- Number of items processed
- Guardrail results (if any)

**2. Runner.run_streamed() - Streaming Response**

Executes the agent and streams the response token by token (word by word):

```Python
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are apples?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True
```

**Returns**: A stream object that yields events in real-time, useful for showing progressive responses to the user.

Creamos un agente sencillo de nombre "Nutrition Assistant"

In [ ]:
# Creamos el agente
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant that provides nutritional information about food items.
    """
    )


Ejecutemos el agente con el Runner:

In [ ]:
# Lo ejecutamos asíncronamente. El agente responde a la pregunta sobre cuán saludables son los plátanos.  
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas? Please, translate the answer to Spanish.")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Las bananas son saludables y nutritivas. Puntos clave:
    
    - Nutrientes: altas en potasio, vitamina B6, vitamina C y fibra.
    - Energía: proporcionan carbohidratos simples para un impulso rápido.
    - Bajo en grasa y sin colesterol; contienen proteína en cantidades modestas.
    - Beneficios: apoyar la salud cardíaca, la digestión y la saciedad.
    - Consideraciones: contienen azúcares naturales; ojo con las porciones si controlas la ingesta de carbohidratos o calorías. Un plátano mediano (~118 g) tiene ~105 calorías, 27 g de carbohidratos, 3 g de fibra, 1 g de proteína y 0,4 g de grasa.
    
    En resumen: opción saludable y conveniente como snack o parte de comidas, con moderación según tus objetivos de energía y carbohidratos.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


-------
Podemos hacer que la respuesta sea en modo streaming... es decir token por token y no esperar a tener toda la respuesta.

In [6]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are apples?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Apples are a healthy, nutrient-dense fruit. Key points:

- Nutrients: about 95 calories, 4–5 g fiber (mostly insoluble, some soluble pectin), vitamin C, potassium, and various antioxidants (quercetin, rutin, flavonoids).
- Health benefits: supports heart health, helps with digestion and satiety, may aid weight management, and provides antioxidants that combat inflammation.
- Glycemic impact: low to moderate glycemic load; whole apples have fiber that slows sugar absorption. Apple juice is higher in sugars and lower in fiber.
- Practical tips: eat them whole or cut with the skin on to maximize fiber and nutrients.
- Considerations: wash well or buy organic if pesticides are a concern; watch portions if managing sugar intake; avoid excessive juice/adding sugar.

Bottom line: apples are a healthy, convenient snack that fit most balanced diets. 1–2 apples a day is a common, reasonable amount for many people.